In [ ]:
!pip3 install qdrant-client transformers torch==1.9.0 ipywidgets IProgress sentence-transformers

In [88]:
import sys
sys.path.append('/arrow/python')

import glob
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

from qdrant_client import QdrantClient
from qdrant_openapi_client.models.models import Filter, FieldCondition, Match, Distance

qc = QdrantClient(host="code-book-notebooks", port=6333, timeout=None)

qc.recreate_collection(collection_name="gh-2017", vector_size=384, distance=Distance.COSINE)

In [2]:
nodes = pq.ParquetDataset(
  glob.glob('/mnt/chunk-1/relations/node_info.merged/*.parquet')
).read().to_pandas()

payload = nodes[['text', 'type', 'gid', 'fid']].to_dict('records')

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Load
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# Encode
vectors = nodes.text.apply(lambda x: model.encode(x))
# Reshape a bit
vectors_arr = np.array(vectors.values.tolist())


In [ ]:
qc.upload_collection(
    collection_name="gh-2017",
    vectors=vectors_arr,
    payload=payload,
    ids=None,
    batch_size=1024,
    parallel=16
)

In [ ]:
results = qc.search(
  collection_name="gh-2017",
  query_vector=model.encode("secretive"),
  top=100,
  query_filter=Filter(must=[FieldCondition(key='type', match=Match(keyword="string"))])
)

[ y['text'] for (x,y) in results if y != {} ]
# results